# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [57]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_Vacation.csv")

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,17,Cape Town,ZA,80.11,-33.9258,18.4232,Townhouse Hotel
1,20,Sinnamary,GF,82.22,5.3833,-52.9500,Hôtel du fleuve
2,21,Hambantota,LK,77.34,6.1241,81.1185,Oasis Aurweda Resort Hambathota
3,30,Port Elizabeth,ZA,78.10,-33.9180,25.5701,Waterford Hotel
4,31,Nacala,MZ,83.52,-14.5428,40.6728,Indico Apart Hotel 2


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [6]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
    frame_width = 700, 
    frame_height = 500, 
    size = "Max Temp", 
    scale = 1, 
    color = "City", 
    hover_cols =["City", "Hotel Name", "Country"])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [9]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"]== "Salinopolis"]
vacation_end = vacation_df.loc[vacation_df["City"]== "Itarema"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]== "Joao Pessoa"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]== "Maceio"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"]== "Salinopolis"] 

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [10]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3,],ignore_index=True)
# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,568,Salinopolis,BR,84.25,-0.6136,-47.3561,Hotel Salinópolis
1,377,Joao Pessoa,BR,82.63,-7.1150,-34.8631,Hotel Guarany
2,466,Maceio,BR,85.44,-9.6658,-35.7353,Maceió Hostel e Pousada
3,568,Salinopolis,BR,84.25,-0.6136,-47.3561,Hotel Salinópolis


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [11]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()
# Display sample data
waypoints_df

,Lat,Lng
0,-0.6136,-47.3561
1,-7.1150,-34.8631
2,-9.6658,-35.7353
3,-0.6136,-47.3561


## Use GeoViews to create map that shows the four cities in the itinerary

In [49]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
    frame_width = 700, 
    frame_height = 800, 
    scale = 1, 
    color = "red", 
    hover_cols =["City"])

In [50]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [17]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [18]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '981352fff39b478d98e3211e0f68f143',
 'waypoints': '-0.6136,-47.3561|-7.115,-34.8631|-9.6658,-35.7353|-0.6136,-47.3561'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [19]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)
# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [51]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [63]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for step in legs:
    for direction in step: 
        lat = direction[1]
        lng = direction[0]
        longitude.append(lng)
        latitude.append(lat)

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [64]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["Longitude"]=longitude
route_df["latitude"]=latitude

# Display sample data
route_df.head()

,Longitude,latitude
0,-47.356058,-0.613616
1,-47.356157,-0.613876
2,-47.356385,-0.614667
3,-47.356641,-0.615602
4,-47.356746,-0.616000


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [70]:
# Configure the route path by using the GeoViews' Path function
route_path = route_df.hvplot.points(
    "Longitude", 
    "latitude", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 700, 
    frame_height = 800,
    color = "red",
    line_width = 3,
    alpha = 1)

In [71]:
# Display a composed plot by using the route_map and route_path objects
route_path*itinerary_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    color = "City", 
    size = 100,
    frame_width = 700, 
    frame_height = 800, 
    alpha = 0.5)

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Points.I  :Points   [Longitude,latitude]
   .Points.II :Points   [Lng,Lat]   (City)